In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('lr1').getOrCreate()

In [2]:
data=spark.read.csv('titanic.csv',inferSchema=True,header=True)

In [3]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [4]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [6]:
data.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

In [8]:
data.show(3)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 3 rows



In [11]:
data=data.select(['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare','Embarked'])

In [13]:
my_final_data=data.na.drop()

In [15]:
my_final_data.summary().show()

+-------+------------------+------------------+------+-----------------+------------------+-------------------+------------------+--------+
|summary|          Survived|            Pclass|   Sex|              Age|             SibSp|              Parch|              Fare|Embarked|
+-------+------------------+------------------+------+-----------------+------------------+-------------------+------------------+--------+
|  count|               712|               712|   712|              712|               712|                712|               712|     712|
|   mean|0.4044943820224719| 2.240168539325843|  null|29.64209269662921|0.5140449438202247|0.43258426966292135| 34.56725140449432|    null|
| stddev|0.4911389472541192|0.8368543166903446|  null|14.49293290032352|0.9306921267673427| 0.8541814457454133|52.938648174710906|    null|
|    min|                 0|                 1|female|             0.42|                 0|                  0|               0.0|       C|
|    25%|           

In [16]:
my_final_data.show()

+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|
|       1|     1|female|38.0|    1|    0|71.2833|       C|
|       1|     3|female|26.0|    0|    0|  7.925|       S|
|       1|     1|female|35.0|    1|    0|   53.1|       S|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|
|       0|     1|  male|54.0|    0|    0|51.8625|       S|
|       0|     3|  male| 2.0|    3|    1| 21.075|       S|
|       1|     3|female|27.0|    0|    2|11.1333|       S|
|       1|     2|female|14.0|    1|    0|30.0708|       C|
|       1|     3|female| 4.0|    1|    1|   16.7|       S|
|       1|     1|female|58.0|    0|    0|  26.55|       S|
|       0|     3|  male|20.0|    0|    0|   8.05|       S|
|       0|     3|  male|39.0|    1|    5| 31.275|       S|
|       0|     3|female|14.0|    0|    0| 7.8542|       

In [21]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer)
from pyspark.ml.classification import LogisticRegression
gender_indexer=StringIndexer(inputCol='Sex',outputCol='SexIndex')
gender_encoder=OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')
embark_indexer=StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder=OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')
assembler=VectorAssembler(inputCols=['Pclass',
 'SexVec',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'EmbarkVec'],outputCol='features')
log_reg_titanic=LogisticRegression(featuresCol='features',labelCol='Survived')

In [23]:
from pyspark.ml import Pipeline
pipeline=Pipeline(stages=[gender_indexer,embark_indexer,
                         gender_encoder,embark_encoder,
                         assembler,log_reg_titanic])
train_data,test_data=my_final_data.randomSplit([0.7,0.3])
fit_model=pipeline.fit(train_data)

In [24]:
results=fit_model.transform(test_data)

In [31]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval=BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

In [29]:
results.select('Survived,'preshow()

+--------+------+------+----+-----+-----+-------+--------+--------+-----------+-------------+-------------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|SexIndex|EmbarkIndex|       SexVec|    EmbarkVec|            features|       rawPrediction|         probability|prediction|
+--------+------+------+----+-----+-----+-------+--------+--------+-----------+-------------+-------------+--------------------+--------------------+--------------------+----------+
|       0|     1|female| 2.0|    1|    2| 151.55|       S|     1.0|        0.0|    (1,[],[])|(2,[0],[1.0])|[1.0,0.0,2.0,1.0,...|[-4.0245342487803...|[0.01755795469605...|       1.0|
|       0|     1|  male|19.0|    3|    2|  263.0|       S|     0.0|        0.0|(1,[0],[1.0])|(2,[0],[1.0])|[1.0,1.0,19.0,3.0...|[-0.3230605406826...|[0.41993005225230...|       1.0|
|       0|     1|  male|28.0|    0|    0|   47.1|       S|     0.0|        0.0|(1,[0],[1.0